# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-33b80657ad-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString
    
    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("IT Companies explorative search") 

Consider the following exploratory information need:

> Compare companies across different sectors in U.K., U.S., and Canada, consider number of employees, companies owned or acquired, and revenue or assets

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wd:Q4830453`  | Business      | node      |
| `wd:Q13977`    | Bloomberg L.P.| node |
| `wd:Q502121`   | BlackBerry    | node |
| `wd:Q16`        | Canada        | node |
| `wd:Q145`      | U.K.          | node |
| `wd:Q30`       | U.S.A.        | node |


Also consider

```
?p wdt:P17 wd:Q16  . 
?p wdt:P31 wd:Q4830453  . 
```

is the BGP to retrieve all **canadian businesses**

## Workload Goals

1. Identify the BGP for obtaining number of employees of a company and other relevant numerical attributes

2. Identify the BGP to retrieve all companies owned by a company

3. Is there some company that owns companies in other countries?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of employees  and other relevant numeric attributes
 
   5.1 What are the top-10 companies for a given attribute?
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each category and country?
   
   5.3 Which business in each country owns more businesses in other countries?


In [5]:
#testing query to get name of properties and classes
q = f"""
select ?name where{{
wdt:P17 <http://schema.org/name> ?name
}}
"""
run_query(q)

[('name', 'country')]


1

In [6]:
#get all canadian businesses (with name)
#limit output to avoid scrolling too much
q = f"""
select ?business ?name where{{
?business wdt:P17 wd:Q16 ;
    wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?name
}}
limit(10)
"""
run_query(q)

[('business', 'http://www.wikidata.org/entity/Q17107879'), ('name', 'Martin & Vleminckx')]
[('business', 'http://www.wikidata.org/entity/Q3347600'), ('name', 'OMERS')]
[('business', 'http://www.wikidata.org/entity/Q4724292'), ('name', 'Algoma Central')]
[('business', 'http://www.wikidata.org/entity/Q1647539'), ('name', 'Royal Canadian Mint')]
[('business', 'http://www.wikidata.org/entity/Q17012723'), ('name', 'Nova Scotia Light and Power')]
[('business', 'http://www.wikidata.org/entity/Q3499025'), ('name', 'Fresh TV')]
[('business', 'http://www.wikidata.org/entity/Q6934764'), ('name', 'Multimatic Motorsports')]
[('business', 'http://www.wikidata.org/entity/Q10439578'), ('name', 'Calgary Sports and Entertainment')]
[('business', 'http://www.wikidata.org/entity/Q7059529'), ('name', 'Northland Power')]
[('business', 'http://www.wikidata.org/entity/Q29123207'), ('name', 'Stryker (Canada)')]


10

In [7]:
#we want to find the uri and the name for UK, US, Canada to focus our exploratory search on those 3
q = f"""
select ?country ?name where{{
?country wdt:P31 wdt:P17 ;
    <http://schema.org/name> ?name .
filter regex(?name, '.*(uk|us|Canada|UK|US|U.K|U.S|United Kingdom|United States).*')
}}
"""
run_query(q)

Empty


0

In [8]:
#probably something wrong with the regex, let's try and find only Canada even if we already have its uri
q = f"""
select ?country ?name where{{
?country wdt:P31 wdt:P17 ;
    <http://schema.org/name> ?name .
filter regex(?name, '.*(C|c)(anada|ANADA).*')
}}
"""
run_query(q)

Empty


0

In [9]:
#let's try this instead: get 10 countries with their name
q = f"""
select ?country ?name where{{
?country wdt:P31 wdt:P17 ;
    <http://schema.org/name> ?name .
}}
limit(10)
"""
run_query(q)

Empty


0

In [10]:
#let's try and get only countries without their name
q = f"""
select ?country where{{
?country wdt:P31 wdt:P17
}}
limit(10)
"""
run_query(q)

Empty


0

In [11]:
#try and run a query that gave us results before
q = f"""
select ?business ?name where{{
?business wdt:P17 wd:Q16 ;
    wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?name
}}
limit(10)
"""
run_query(q)

[('business', 'http://www.wikidata.org/entity/Q17107879'), ('name', 'Martin & Vleminckx')]
[('business', 'http://www.wikidata.org/entity/Q3347600'), ('name', 'OMERS')]
[('business', 'http://www.wikidata.org/entity/Q4724292'), ('name', 'Algoma Central')]
[('business', 'http://www.wikidata.org/entity/Q1647539'), ('name', 'Royal Canadian Mint')]
[('business', 'http://www.wikidata.org/entity/Q17012723'), ('name', 'Nova Scotia Light and Power')]
[('business', 'http://www.wikidata.org/entity/Q3499025'), ('name', 'Fresh TV')]
[('business', 'http://www.wikidata.org/entity/Q6934764'), ('name', 'Multimatic Motorsports')]
[('business', 'http://www.wikidata.org/entity/Q10439578'), ('name', 'Calgary Sports and Entertainment')]
[('business', 'http://www.wikidata.org/entity/Q7059529'), ('name', 'Northland Power')]
[('business', 'http://www.wikidata.org/entity/Q29123207'), ('name', 'Stryker (Canada)')]


10

In [12]:
#i got confused: wdt:P17 is a property, not a class
#let's try again and get US, UK, Canada as we wanted
q = f"""
select distinct ?country ?name where{{
    ?subject wdt:P17 ?country .
    ?country <http://schema.org/name> ?name .
    filter regex(?name, '.*(uk|us|Canada|UK|US|U.K|U.S|United Kingdom|United States).*')
}}

"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q3088906'), ('name', 'France–United Kingdom border')]
[('country', 'http://www.wikidata.org/entity/Q106577'), ('name', 'Neustria')]
[('country', 'http://www.wikidata.org/entity/Q208169'), ('name', 'Republic of Ragusa')]
[('country', 'http://www.wikidata.org/entity/Q5677983'), ('name', 'Indigenous reservation')]
[('country', 'http://www.wikidata.org/entity/Q1061488'), ('name', 'United States of Colombia')]
[('country', 'http://www.wikidata.org/entity/Q11285099'), ('name', 'United States occupation of the Ryukyu Islands')]
[('country', 'http://www.wikidata.org/entity/Q76236'), ('name', 'Lithuanian–Belorussian Soviet Socialist Republic')]
[('country', 'http://www.wikidata.org/entity/Q19128339'), ('name', 'American Russians')]
[('country', 'http://www.wikidata.org/entity/Q12631'), ('name', 'Meuse')]
[('country', 'http://www.wikidata.org/entity/Q139319'), ('name', 'Russian Republic')]
[('country', 'http://www.wikidata.org/entity/Q190967'), ('name

179

In [14]:
#let's try and find the uri for the the class Country using the uri for Canada, already provided
q = f"""
select ?o ?name where{{
wd:Q16 wdt:P31 ?o .
?o <http://schema.org/name> ?name
}}
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]


2

In [15]:
#now we can easily isolate instances of country
class_country = 'wd:Q6256'
#let's try and find again the uris for Canada, United States, United Kingdom
q = f"""
select ?country ?name where{{
?country wdt:P31 wd:Q6256 ;
    <http://schema.org/name> ?name
filter regex(?name, '(united kingdom|canada|united states of america)' ,'i')
}}

"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]


3

In [17]:
#let's try and get all of the properties associated with a business, with their names
q = f"""
select distinct ?property ?propertyName where{{
?b wdt:P31 wd:Q4830453 .
?b ?property ?predicate .
?property <http://schema.org/name> ?propertyName
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P4446'), ('propertyName', 'reward program')]
[('property', 'http://www.wikidata.org/prop/direct/P1268'), ('propertyName', 'represents')]
[('property', 'http://www.wikidata.org/prop/direct/P2751'), ('propertyName', 'Roller Coaster Database ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3376'), ('propertyName', 'Enterprise number (Belgium)')]
[('property', 'http://www.wikidata.org/prop/direct/P6617'), ('propertyName', 'RoMEO publisher ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7403'), ('propertyName', 'Publons publisher ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7972'), ('propertyName', 'CVR person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8562'), ('propertyName', 'JSTOR publisher ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5505'), ('propertyName', 'WorldCat Registry ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7639'), ('propertyName', 'OverDrive publishe

1659

In [18]:
#let's try instead to identify properties related only to companies in us, uk, canada
q = f"""
select distinct ?property ?propertyName where{{
?b wdt:P31 wd:Q4830453 .
?b ?property ?predicate .
?b wdt:P17 ?country .
filter(?country in (wd:Q16, wd:Q145, wd:Q30))
?property <http://schema.org/name> ?propertyName .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P2751'), ('propertyName', 'Roller Coaster Database ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1056'), ('propertyName', 'product or material produced')]
[('property', 'http://www.wikidata.org/prop/direct/P112'), ('propertyName', 'founded by')]
[('property', 'http://www.wikidata.org/prop/direct/P1128'), ('propertyName', 'employees')]
[('property', 'http://www.wikidata.org/prop/direct/P127'), ('propertyName', 'owned by')]
[('property', 'http://www.wikidata.org/prop/direct/P1454'), ('propertyName', 'legal form')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('propertyName', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P159'), ('propertyName', 'headquarters location')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('propertyName', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('propertyName', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P20

1066

In [19]:
#let's try and filter the properties by name, we're looking for keywords such as number of employees, revenue, capital, assets
#sector, if a company owns or is onwned by some other company, the ceo...
q = f"""
select distinct ?property ?propertyName where{{
?b wdt:P31 wd:Q4830453 .
?b ?property ?predicate .
?b wdt:P17 ?country .
filter(?country in (wd:Q16, wd:Q145, wd:Q30)) .
?property <http://schema.org/name> ?propertyName .
filter regex(?propertyName, '.*(employee|revenue|capital|asset|owne|ceo|c.e.o.|sector).*' ,'i') .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1128'), ('propertyName', 'employees')]
[('property', 'http://www.wikidata.org/prop/direct/P127'), ('propertyName', 'owned by')]
[('property', 'http://www.wikidata.org/prop/direct/P1830'), ('propertyName', 'owner of')]
[('property', 'http://www.wikidata.org/prop/direct/P2139'), ('propertyName', 'total revenue')]
[('property', 'http://www.wikidata.org/prop/direct/P2403'), ('propertyName', 'total assets')]
[('property', 'http://www.wikidata.org/prop/direct/P2226'), ('propertyName', 'market capitalization')]
[('property', 'http://www.wikidata.org/prop/direct/P4103'), ('propertyName', 'assets under management')]
[('property', 'http://www.wikidata.org/prop/direct/P2663'), ('propertyName', 'common equity tier 1 capital ratio (CETI)')]
[('property', 'http://www.wikidata.org/prop/direct/P4195'), ('propertyName', 'category for employees of the organization')]
[('property', 'http://www.wikidata.org/prop/direct/P3087'), ('propertyName', 'fiscal/t

10

In [20]:
#let's define for now variables for the properties we care about
owned_by = 'wdt:P127'
owner_of = 'wdt:P1830'
n_employees = 'wdt:P1128'
total_revenue = 'wdt:P2139'
total_assets = 'wdt:P2403'
#we couldn't find a property related to sector, let's try with a company we know, for example Microsoft, and see its properties
q = f"""
select distinct ?companyName ?property ?propertyName where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?companyName ;
    wdt:P17 wd:Q30 ;
    ?property ?o .
?property <http://schema.org/name> ?propertyName .
filter regex(?companyName, 'microsoft', 'i')
}}
"""
run_query(q)

[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P8060'), ('propertyName', 'Glassdoor company ID')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P8939'), ('propertyName', 'Google Play developer slug')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P1005'), ('propertyName', 'Portuguese National Library ID')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P1037'), ('propertyName', 'director / manager')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P1056'), ('propertyName', 'product or material produced')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P112'), ('propertyName', 'founded by')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/direct/P1128'), ('propertyName', 'employees')]
[('companyName', 'Microsoft'), ('property', 'http://www.wikidata.org/prop/di

149

In [21]:
#with a more readable output, we're able to find out the properties related to the CEO and the "product or material produced"
ceo = 'wdt:P169'
produces = 'wdt:P1056'
#i forgot to show the company uri, i have to filter by name of the company again
#let's find out the ceo and the product of microsoft
q = f"""
select ?company, ?companyName, ?ceoName, ?product, ?productName where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P17 wd:Q30 ;
    wdt:P169 ?ceo ;
    wdt:P1056 ?product ;
    <http://schema.org/name> ?companyName .
?ceo <http://schema.org/name> ?ceoName .
?product <http://schema.org/name> ?productName .
filter regex(?companyName, 'microsoft', 'i')
}}
"""
run_query(q)

[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft'), ('ceoName', 'Satya Nadella'), ('product', 'http://www.wikidata.org/entity/Q132020'), ('productName', 'Xbox')]
[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft'), ('ceoName', 'Satya Nadella'), ('product', 'http://www.wikidata.org/entity/Q1071340'), ('productName', 'Microsoft Servers')]
[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft'), ('ceoName', 'Satya Nadella'), ('product', 'http://www.wikidata.org/entity/Q134067'), ('productName', 'Microsoft Visual Studio')]
[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft'), ('ceoName', 'Satya Nadella'), ('product', 'http://www.wikidata.org/entity/Q40984'), ('productName', 'Skype')]
[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft'), ('ceoName', 'Satya Nadella'), ('product', 'http://www.wikidata.org/entity/Q182496'), ('productName', 'Micro

14

In [22]:
#there is one product, "computer hardware", that may help us find the class related to products of a company, the other results are just the name of specific products
#let's find the class of "computer hardware"
computer_hardware = 'wd:Q3966'
q = f"""
select distinct ?class ?className where{{
wd:Q3966 wdt:P31 ?class .
?class <http://schema.org/name> ?className .
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q11650'), ('className', 'electronics')]


1

In [23]:
#let's do that again for "electronics"
electronics = 'wd:Q11650'
q = f"""
select distinct ?class ?className where{{
wd:Q11650 wdt:P31 ?class .
?class <http://schema.org/name> ?className .
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q11862829'), ('className', 'academic discipline')]
[('class', 'http://www.wikidata.org/entity/Q2465832'), ('className', 'branch of science')]


2

In [24]:
#this is bringing us nowhere, maybe "computer hardware" is subclass of something more general
subclass = 'wdt:P279'
q = f"""
select distinct ?super ?name where{{
wd:Q3966 wdt:P279 ?super .
?super <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('super', 'http://www.wikidata.org/entity/Q11661'), ('name', 'information technology')]
[('super', 'http://www.wikidata.org/entity/Q2424752'), ('name', 'product')]
[('super', 'http://www.wikidata.org/entity/Q2858615'), ('name', 'electronic machine')]


3

In [25]:
#let's try to fetch american companies and their products
product = 'wd:Q2424752'
q = f"""
select ?companyName ?productName where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1056 ?product ;
    <http://schema.org/name> ?companyName ;
    wdt:P17 wd:Q30 .
?product wdt:P279 wd:Q2424752 ;
    <http://schema.org/name> ?productName .
    
}}
order by asc(?productName)
limit 20
"""
run_query(q)

[('companyName', 'Zagg'), ('productName', 'Screen protector')]
[('companyName', 'Balsam Hill'), ('productName', 'artificial Christmas tree')]
[('companyName', 'Timken Company'), ('productName', 'bearing')]
[('companyName', 'H. H. Franklin Manufacturing Company'), ('productName', 'bearing')]
[('companyName', 'Hamilton Shirts'), ('productName', 'bespoke')]
[('companyName', 'Diversey, Inc.'), ('productName', 'cleaning product')]
[('companyName', 'Oakley, Inc.'), ('productName', 'clothing')]
[('companyName', 'Columbia Sportswear Company'), ('productName', 'clothing')]
[('companyName', 'TRU-SPEC'), ('productName', 'clothing')]
[('companyName', 'Charming Shoppes'), ('productName', 'clothing')]
[('companyName', 'Banana Republic'), ('productName', 'clothing')]
[('companyName', 'Warnaco Group'), ('productName', 'clothing')]
[('companyName', 'Spanx, Inc.'), ('productName', 'clothing')]
[('companyName', 'Vapor Apparel'), ('productName', 'clothing')]
[('companyName', 'New Era Cap Company'), ('prod

20

In [26]:
#with the more general product class that we found, does microsoft produce software as well?
#let's get microsoft's uri
q = f"""
select distinct ?company ?companyName where{{
?company wdt:P31 wd:Q4830453;
    <http://schema.org/name> ?companyName .
filter regex(?companyName, 'microsoft', 'i')
}}
"""
run_query(q)

[('company', 'http://www.wikidata.org/entity/Q20081886'), ('companyName', 'Microsoft (Ukraine)')]
[('company', 'http://www.wikidata.org/entity/Q2283'), ('companyName', 'Microsoft')]
[('company', 'http://www.wikidata.org/entity/Q6840157'), ('companyName', 'Microsoft Japan')]
[('company', 'http://www.wikidata.org/entity/Q6840192'), ('companyName', 'Microsoft Production Studios')]
[('company', 'http://www.wikidata.org/entity/Q16538568'), ('companyName', 'Microsoft Mobile')]
[('company', 'http://www.wikidata.org/entity/Q30273752'), ('companyName', 'Microsoft (France)')]
[('company', 'http://www.wikidata.org/entity/Q31327320'), ('companyName', 'Microsoft (cz)')]
[('company', 'http://www.wikidata.org/entity/Q6840202'), ('companyName', 'Microsoft Research Asia')]
[('company', 'http://www.wikidata.org/entity/Q80054718'), ('companyName', 'Microsoft Ireland Research')]
[('company', 'http://www.wikidata.org/entity/Q80054807'), ('companyName', 'Microsoft Global Finance')]
[('company', 'http://www.

25

In [27]:
company_microsoft = 'wd:Q2283'
#let's see all products of microsoft
q = f"""
select distinct ?product ?productName where{{
wd:Q2283 wdt:P1056 ?product .
?product wdt:P279 wd:Q2424752 .
?product <http://schema.org/name> ?productName
}}
"""
run_query(q)

[('product', 'http://www.wikidata.org/entity/Q3966'), ('productName', 'computer hardware')]


1

In [28]:
#let's check visual studio's class and superclass
visual_studio = 'wd:Q134067'
q = f"""
select distinct ?class ?className ?superClass ?superClassName where{{
wd:Q134067 wdt:P31 ?class .
wd:Q134067 wdt:P279 ?superClass .
?class <http://schema.org/name> ?className .
?superClass <http://schema.org/name> ?superClassName .
}}
"""
run_query(q)

Empty


0

In [29]:
#let's add optionals
q = f"""
select distinct ?class ?className ?superClass ?superClassName where{{
wd:Q134067 wdt:P31 ?class .
    optional{{
        wd:Q134067 wdt:P279 ?superClass .
        ?superClass <http://schema.org/name> ?superClassName .
    }}
?class <http://schema.org/name> ?className .
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q13741'), ('className', 'integrated development environment')]


1

In [30]:
#this didn't help us either. the only thing i can think of is to go back to the list of properties for microsoft and search for another one
#doing this i found the property "industry", let's check that out
industry = 'wdt:P452'
#let's see the industry for some companies in the us
q = f"""
select ?companyName ?industry ?industryName where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P17 wd:Q30 ;
    wdt:P452 ?industry ;
    <http://schema.org/name> ?companyName .
?industry <http://schema.org/name> ?industryName
}}
limit 20
"""
run_query(q)

[('companyName', 'Lyft'), ('industry', 'http://www.wikidata.org/entity/Q41222493'), ('industryName', 'taxi service')]
[('companyName', 'Marvel Productions'), ('industry', 'http://www.wikidata.org/entity/Q375336'), ('industryName', 'film studio')]
[('companyName', 'Walt Disney Television'), ('industry', 'http://www.wikidata.org/entity/Q10689397'), ('industryName', 'television production company')]
[('companyName', 'Far East Broadcasting Company'), ('industry', 'http://www.wikidata.org/entity/Q15078788'), ('industryName', 'broadcasting')]
[('companyName', 'American Electric Power'), ('industry', 'http://www.wikidata.org/entity/Q383973'), ('industryName', 'electricity generation')]
[('companyName', 'American Locomotive Company'), ('industry', 'http://www.wikidata.org/entity/Q4265939'), ('industryName', 'locomotive manufacturing')]
[('companyName', 'Commodore International'), ('industry', 'http://www.wikidata.org/entity/Q581105'), ('industryName', 'consumer electronics')]
[('companyName', 

20

In [31]:
#let's count the number of companies in the us for each industry
q = f"""
select ?industry ?industryName (count(*) as ?total) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P452 ?industry ;
    wdt:P17 wd:Q30 .
    
?industry <http://schema.org/name> ?industryName .
}}
group by ?industry ?industryName
limit 30
"""
run_query(q)

[('industry', 'http://www.wikidata.org/entity/Q219409'), ('industryName', 'venture capital'), ('total', '2')]
[('industry', 'http://www.wikidata.org/entity/Q484847'), ('industryName', 'e-commerce'), ('total', '51')]
[('industry', 'http://www.wikidata.org/entity/Q11650'), ('industryName', 'electronics'), ('total', '61')]
[('industry', 'http://www.wikidata.org/entity/Q31207'), ('industryName', 'health care'), ('total', '101')]
[('industry', 'http://www.wikidata.org/entity/Q1481411'), ('industryName', 'IT service management'), ('total', '14')]
[('industry', 'http://www.wikidata.org/entity/Q3510521'), ('industryName', 'computer security'), ('total', '27')]
[('industry', 'http://www.wikidata.org/entity/Q1416012'), ('industryName', 'managed care'), ('total', '6')]
[('industry', 'http://www.wikidata.org/entity/Q775325'), ('industryName', 'transportation engineering'), ('total', '1')]
[('industry', 'http://www.wikidata.org/entity/Q160738'), ('industryName', 'role-playing game'), ('total', '8')

30

## Copy of goals to avoid scrolling up

1. Identify the BGP for obtaining number of employees of a company and other relevant numerical attributes

2. Identify the BGP to retrieve all companies owned by a company

3. Is there some company that owns companies in other countries?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of employees  and other relevant numeric attributes
 
   5.1 What are the top-10 companies for a given attribute?
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each category and country?
   
   5.3 Which business in each country owns more businesses in other countries?


In [32]:
#let's start answering to the goals using all the properties found earlier
#point 1 is already complete since we know all the relevant properties. Let's list them for some companies

#employees
q = f"""
select distinct ?companyName ?employees where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?companyName ;
    wdt:P1128 ?employees .
}}
limit 20
"""
run_query(q)

[('companyName', 'APM Terminals B.V'), ('employees', '20600')]
[('companyName', 'Agrostroj Pelhřimov'), ('employees', '2456')]
[('companyName', 'Cengage Learning'), ('employees', '5500')]
[('companyName', 'DHL'), ('employees', '570000')]
[('companyName', 'Elevation Partners'), ('employees', '20')]
[('companyName', 'DNB Bank'), ('employees', '9020')]
[('companyName', 'R. J. Reynolds Tobacco Company'), ('employees', '400000')]
[('companyName', 'Merrill Lynch'), ('employees', '60000')]
[('companyName', 'Gillette'), ('employees', '29000')]
[('companyName', 'KCA DEUTAG'), ('employees', '8000')]
[('companyName', 'Česká zbrojovka Strakonice'), ('employees', '1244')]
[('companyName', 'Nobiskrug'), ('employees', '1000')]
[('companyName', 'Mt.Gox'), ('employees', '19')]
[('companyName', 'Schypke'), ('employees', '80')]
[('companyName', 'FARYS'), ('employees', '774')]
[('companyName', 'VeriSilicon Holdings Co., Ltd.'), ('employees', '400')]
[('companyName', 'Remondis'), ('employees', '32000')]
[(

20

In [33]:
#assets and revenue
q = f"""
select distinct ?companyName ?assets ?revenue where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?companyName ;
    wdt:P2403 ?assets ;
    wdt:P2139 ?revenue .
}}
limit 20
"""
run_query(q)

[('companyName', 'Agrostroj Pelhřimov'), ('assets', '7824063000'), ('revenue', '6186466000')]
[('companyName', 'DNB Bank'), ('assets', '2793294000000'), ('revenue', '54857000000')]
[('companyName', 'Merrill Lynch'), ('assets', '603000000000'), ('revenue', '20217000000')]
[('companyName', 'Merrill Lynch'), ('assets', '603000000000'), ('revenue', '24397000000')]
[('companyName', 'Česká zbrojovka Strakonice'), ('assets', '2165963000'), ('revenue', '1462114000')]
[('companyName', 'Energetický a průmyslový holding'), ('assets', '13329000000'), ('revenue', '6998000000')]
[('companyName', 'Energetický a průmyslový holding'), ('assets', '342889000000'), ('revenue', '6998000000')]
[('companyName', 'Energetický a průmyslový holding'), ('assets', '13329000000'), ('revenue', '180024000000')]
[('companyName', 'Energetický a průmyslový holding'), ('assets', '342889000000'), ('revenue', '180024000000')]
[('companyName', 'AGEL'), ('assets', '10518247000'), ('revenue', '18246784000')]
[('companyName', 

20

In [34]:
#combining everything
q = f"""
select ?company ?companyName ?n_employees ?revenue ?assets where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?companyName ;
    wdt:P1128 ?n_employees ;
    wdt:P2139 ?revenue ;
    wdt:P2403 ?assets .
}}
order by desc(?revenue)
limit(10)
"""
run_query(q)

[('company', 'http://www.wikidata.org/entity/Q20718'), ('companyName', 'Samsung Electronics'), ('n_employees', '392305'), ('revenue', '201870000000000'), ('assets', '262174300000000')]
[('company', 'http://www.wikidata.org/entity/Q53268'), ('companyName', 'Toyota'), ('n_employees', '359542'), ('revenue', '28403118000000'), ('assets', '47427597000000')]
[('company', 'http://www.wikidata.org/entity/Q201653'), ('companyName', 'SoftBank'), ('n_employees', '68402'), ('revenue', '9158765000000'), ('assets', '31180466000000')]
[('company', 'http://www.wikidata.org/entity/Q53247'), ('companyName', 'Panasonic Corporation'), ('n_employees', '257533'), ('revenue', '7343707000000'), ('assets', '5982961000000')]
[('company', 'http://www.wikidata.org/entity/Q102673'), ('companyName', 'Gazprom'), ('n_employees', '469600'), ('revenue', '6321559000000'), ('assets', '23352185000000')]
[('company', 'http://www.wikidata.org/entity/Q1141123'), ('companyName', 'Rosneft'), ('n_employees', '334600'), ('revenu

10

In [35]:
#we can also answer goal 2
#let's select Microsoft as owner
q = f"""
select distinct ?owned ?name where{{
wd:Q2283 wdt:P1830 ?owned .
?owned <http://schema.org/name> ?name ;
    wdt:P31 wd:Q4830453 .
}}
"""
run_query(q)

[('owned', 'http://www.wikidata.org/entity/Q3073996'), ('name', 'Xamarin')]
[('owned', 'http://www.wikidata.org/entity/Q213660'), ('name', 'LinkedIn')]
[('owned', 'http://www.wikidata.org/entity/Q246125'), ('name', 'Skype Technologies')]
[('owned', 'http://www.wikidata.org/entity/Q131910'), ('name', 'Yammer')]
[('owned', 'http://www.wikidata.org/entity/Q224599'), ('name', 'Powerset')]
[('owned', 'http://www.wikidata.org/entity/Q31327320'), ('name', 'Microsoft (cz)')]
[('owned', 'http://www.wikidata.org/entity/Q4048220'), ('name', 'Revolution Analytics')]
[('owned', 'http://www.wikidata.org/entity/Q6715783'), ('name', 'MGS Mobile Gaming')]
[('owned', 'http://www.wikidata.org/entity/Q6784797'), ('name', 'Massive Incorporated')]
[('owned', 'http://www.wikidata.org/entity/Q958338'), ('name', 'Microsoft TechNet')]


10

In [36]:
#now let's see the country of companies owned by Microsoft
q = f"""
select distinct ?name ?countryName where{{
wd:Q2283 wdt:P1830 ?owned .
?owned <http://schema.org/name> ?name ;
    wdt:P31 wd:Q4830453 ;
    wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
}}
"""
run_query(q)

[('name', 'Xamarin'), ('countryName', 'United States of America')]
[('name', 'LinkedIn'), ('countryName', 'United States of America')]
[('name', 'Skype Technologies'), ('countryName', 'Luxembourg')]
[('name', 'Yammer'), ('countryName', 'United States of America')]
[('name', 'Powerset'), ('countryName', 'United States of America')]
[('name', 'Microsoft (cz)'), ('countryName', 'Czech Republic')]
[('name', 'Revolution Analytics'), ('countryName', 'United States of America')]


7

In [37]:
# let's try now to run the query "get (some) companies that own other companies in a different country"
q = f"""
select ?companyName ?companyCountry ?ownedCompany ?countryOfOwned where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?companyName ;
    wdt:P17 ?country ;
    wdt:P1830 ?owned .
    
?owned wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?ownedCompany ;
    wdt:P17 ?c .
    
?country <http://schema.org/name> ?companyCountry .
?c <http://schema.org/name> ?countryOfOwned .
filter(?country != ?c) .
filter(?country in (wd:Q16, wd:Q145, wd:Q30))
}}
limit(10)
"""
run_query(q)

[('companyName', 'Gillette'), ('companyCountry', 'United States of America'), ('ownedCompany', 'Braun'), ('countryOfOwned', 'Germany')]
[('companyName', 'Arriva'), ('companyCountry', 'United Kingdom'), ('ownedCompany', 'Società Italiana Autoservizi'), ('countryOfOwned', 'Italy')]
[('companyName', 'Arriva'), ('companyCountry', 'United Kingdom'), ('ownedCompany', 'SAB Autoservizi'), ('countryOfOwned', 'Italy')]
[('companyName', 'Arriva'), ('companyCountry', 'United Kingdom'), ('ownedCompany', 'SAL Lecco'), ('countryOfOwned', 'Italy')]
[('companyName', 'Best Buy'), ('companyCountry', 'United States of America'), ('ownedCompany', 'Future Shop'), ('countryOfOwned', 'Canada')]
[('companyName', 'Anglo American plc'), ('companyCountry', 'United Kingdom'), ('ownedCompany', 'De Beers'), ('countryOfOwned', 'South Africa')]
[('companyName', 'Gibson'), ('companyCountry', 'United States of America'), ('ownedCompany', 'TEAC Corporation'), ('countryOfOwned', 'Japan')]
[('companyName', 'Sun Capital Par

10

In [38]:
#we have found some, so goal 3 can be answered by this ask query
a = f"""
ask where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?company2 ;
    wdt:P17 ?country .

?company2 wdt:P31 wd:Q4830453 ;
    wdt:P17 ?country2 .

filter(?country != ?country2)
}}
"""
run_ask_query(a)

{'head': {'link': []}, 'boolean': True}

In [39]:
#let's go to goal 4 now, the code for the property "legal_form" was found a while ago and it is
legal_form = 'wdt:P1454'
#let's view some legal forms, just out of curiosity
q = f"""
select distinct ?legalForm ?name where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1454 ?legalForm .
?legalForm <http://schema.org/name> ?name
}}
limit 20
"""
run_query(q)

[('legalForm', 'http://www.wikidata.org/entity/Q20746571'), ('name', 'public company limited by shares')]
[('legalForm', 'http://www.wikidata.org/entity/Q18340464'), ('name', 'sociedade anónima')]
[('legalForm', 'http://www.wikidata.org/entity/Q2624661'), ('name', 'Sp. z o.o.')]
[('legalForm', 'http://www.wikidata.org/entity/Q17990971'), ('name', 'public enterprise')]
[('legalForm', 'http://www.wikidata.org/entity/Q1780031'), ('name', 'Kommanditgesellschaft')]
[('legalForm', 'http://www.wikidata.org/entity/Q1518608'), ('name', 'limited liability company')]
[('legalForm', 'http://www.wikidata.org/entity/Q510785'), ('name', 'association without lucrative purpose')]
[('legalForm', 'http://www.wikidata.org/entity/Q64699475'), ('name', 'spółka akcyjna')]
[('legalForm', 'http://www.wikidata.org/entity/Q16917171'), ('name', 'private limited company')]
[('legalForm', 'http://www.wikidata.org/entity/Q11900271'), ('name', 'valtion liikelaitos')]
[('legalForm', 'http://www.wikidata.org/entity/Q15

20

In [40]:
#now let's answer to goal 4, we count for each legal form the number of companies associated
q = f"""
select ?legalForm ?name (count(*) as ?howMany) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1454 ?legalForm .
?legalForm <http://schema.org/name> ?name .
}}
group by ?name ?legalForm
order by desc (?howMany)
"""
run_query(q)

[('legalForm', 'http://www.wikidata.org/entity/Q15646299'), ('name', 'společnost s ručením omezeným'), ('howMany', '13059')]
[('legalForm', 'http://www.wikidata.org/entity/Q3742494'), ('name', 'akciová společnost'), ('howMany', '5551')]
[('legalForm', 'http://www.wikidata.org/entity/Q134161'), ('name', 'joint-stock company'), ('howMany', '5168')]
[('legalForm', 'http://www.wikidata.org/entity/Q460178'), ('name', 'GmbH'), ('howMany', '2823')]
[('legalForm', 'http://www.wikidata.org/entity/Q15649019'), ('name', 'Spoločnosť s ručením obmedzeným'), ('howMany', '2302')]
[('legalForm', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company'), ('howMany', '1407')]
[('legalForm', 'http://www.wikidata.org/entity/Q15042660'), ('name', 'aksjeselskap'), ('howMany', '1312')]
[('legalForm', 'http://www.wikidata.org/entity/Q1480166'), ('name', 'kabushiki gaisha'), ('howMany', '1228')]
[('legalForm', 'http://www.wikidata.org/entity/Q166280'), ('name', 'S.A.'), ('howMany', '1132')]
[('lega

436

In [41]:
#let's restrict to our 3 target countries
q = f"""
select ?legalForm ?name (count(*) as ?howMany) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1454 ?legalForm ;
    wdt:P17 ?country .
?legalForm <http://schema.org/name> ?name .
filter(?country in (wd:Q16, wd:Q145, wd:Q30))
}}
group by ?name ?legalForm
order by desc (?howMany)
"""
run_query(q)

[('legalForm', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company'), ('howMany', '492')]
[('legalForm', 'http://www.wikidata.org/entity/Q134161'), ('name', 'joint-stock company'), ('howMany', '395')]
[('legalForm', 'http://www.wikidata.org/entity/Q1589009'), ('name', 'privately held company'), ('howMany', '343')]
[('legalForm', 'http://www.wikidata.org/entity/Q5225895'), ('name', 'public limited company'), ('howMany', '256')]
[('legalForm', 'http://www.wikidata.org/entity/Q6832945'), ('name', 'private company limited by shares'), ('howMany', '231')]
[('legalForm', 'http://www.wikidata.org/entity/Q149789'), ('name', 'limited liability company'), ('howMany', '198')]
[('legalForm', 'http://www.wikidata.org/entity/Q783794'), ('name', 'company'), ('howMany', '92')]
[('legalForm', 'http://www.wikidata.org/entity/Q658255'), ('name', 'subsidiary'), ('howMany', '86')]
[('legalForm', 'http://www.wikidata.org/entity/Q167037'), ('name', 'corporation'), ('howMany', '79')]
[('legalF

110

In [42]:
#can a company have more than 1 legal form?
q = f"""
select ?companyName (count(?legalForm) as ?count) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1454 ?legalForm ;
    <http://schema.org/name> ?companyName .
}}
group by ?companyName
order by desc(?count)
limit 10
"""
run_query(q)

[('companyName', 'Norma'), ('count', '4')]
[('companyName', 'Honeywell'), ('count', '3')]
[('companyName', 'Banque Saudi Fransi'), ('count', '3')]
[('companyName', 'China Unicom'), ('count', '3')]
[('companyName', 'Aston Martin Lagonda'), ('count', '3')]
[('companyName', 'Siemens'), ('count', '3')]
[('companyName', 'Zenit'), ('count', '3')]
[('companyName', 'ELBA'), ('count', '3')]
[('companyName', 'Nintendo Australia'), ('count', '3')]
[('companyName', 'Mobileye'), ('count', '3')]


10

## Copy of goals to avoid scrolling

5. Analyze the number of employees  and other relevant numeric attributes  
    5.1 What are the top-10 companies for a given attribute?  
    5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each category and country?  
    5.3 Which business in each country owns more businesses in other countries?

In [43]:
#let's answer 5.1
#top-10 for number of employees between US UK and Canada
q = f"""
select ?name ?employees where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1128 ?employees ;
    wdt:P17 ?country ;
    <http://schema.org/name> ?name .
filter(?country in (wd:Q16, wd:Q30, wd:Q145))
}}
order by desc(?employees)
limit(10)
"""
run_query(q)

[('name', 'Walmart'), ('employees', '2500000')]
[('name', 'Amazon'), ('employees', '1298000')]
[('name', 'G4S'), ('employees', '618260')]
[('name', 'United Parcel Service'), ('employees', '434000')]
[('name', 'Tesco'), ('employees', '423092')]
[('name', 'Yum China'), ('employees', '420000')]
[('name', 'R. J. Reynolds Tobacco Company'), ('employees', '400000')]
[('name', 'FedEx'), ('employees', '400000')]
[('name', 'The Home Depot'), ('employees', '400000')]
[('name', 'Berkshire Hathaway'), ('employees', '360000')]


10

In [44]:
#let's add the country as well
q = f"""
select ?name ?employees ?countryName where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1128 ?employees ;
    wdt:P17 ?country ;
    <http://schema.org/name> ?name .
?country <http://schema.org/name> ?countryName .
filter(?country in (wd:Q16, wd:Q30, wd:Q145))
}}
order by desc(?employees)
limit(10)
"""
run_query(q)

[('name', 'Walmart'), ('employees', '2500000'), ('countryName', 'United States of America')]
[('name', 'Amazon'), ('employees', '1298000'), ('countryName', 'United States of America')]
[('name', 'G4S'), ('employees', '618260'), ('countryName', 'United Kingdom')]
[('name', 'United Parcel Service'), ('employees', '434000'), ('countryName', 'United States of America')]
[('name', 'Tesco'), ('employees', '423092'), ('countryName', 'United Kingdom')]
[('name', 'Yum China'), ('employees', '420000'), ('countryName', 'United States of America')]
[('name', 'R. J. Reynolds Tobacco Company'), ('employees', '400000'), ('countryName', 'United States of America')]
[('name', 'FedEx'), ('employees', '400000'), ('countryName', 'United States of America')]
[('name', 'The Home Depot'), ('employees', '400000'), ('countryName', 'United States of America')]
[('name', 'Berkshire Hathaway'), ('employees', '360000'), ('countryName', 'United States of America')]


10

## Copy of goals to avoid scrolling

5. Analyze the number of employees  and other relevant numeric attributes  
    ~~5.1 What are the top-10 companies for a given attribute?~~  
    5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each category and country?  
    5.3 Which business in each country owns more businesses in other countries?

In [45]:
#to answer goal 5.2 i need more properties
#i could search again those properties among the names i found earlier, let's try and use regex instead this time
q = f"""
select distinct ?property ?name where{{
?company wdt:P31 wd:Q4830453 .
?company ?property ?object .
?property <http://schema.org/name> ?name .
filter regex(?name, '.*(assets|income|market).*') .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P2403'), ('name', 'total assets')]
[('property', 'http://www.wikidata.org/prop/direct/P3362'), ('name', 'operating income')]
[('property', 'http://www.wikidata.org/prop/direct/P7534'), ('name', 'MIC market code')]
[('property', 'http://www.wikidata.org/prop/direct/P3679'), ('name', 'stock market index')]
[('property', 'http://www.wikidata.org/prop/direct/P2226'), ('name', 'market capitalization')]
[('property', 'http://www.wikidata.org/prop/direct/P4103'), ('name', 'assets under management')]
[('property', 'http://www.wikidata.org/prop/direct/P2770'), ('name', 'source of income')]


7

In [46]:
#the properties i need are then
total_assets = 'wdt:P2403'
market_capit = 'wdt:P2226'
op_income = 'wdt:P3362'

#the first part of 5.2 suggests that maybe not all companies have all of the 3 properties, let's try and count those that have that properties
#count first total number of companies
q = f"""
select (count(*) as ?n_companies) where{{
?company wdt:P31 wd:Q4830453
}}
"""
run_query(q)

[('n_companies', '193935')]


1

In [47]:
#let's count now all the companies that have at least one of those properties

#first select each company one time
q = f"""
select distinct ?company where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P2403 | wdt:P2226 | wdt:P3362 ?value .
}}
"""
#count all the companies
q2 = f"""
select (count(*) as ?tot) where{{
    {{
        select distinct ?company where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P2403 | wdt:P2226 | wdt:P3362 ?value .

}}



    }}
}}
"""
run_query(q2)

[('tot', '1516')]


1

## Copy of goals to avoid scrolling

5. Analyze the number of employees  and other relevant numeric attributes  
    ~~5.1 What are the top-10 companies for a given attribute?~~  
    5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each category and country?  
    5.3 Which business in each country owns more businesses in other countries?

In [50]:
#5.2) to answer to "What is the min, max, and average in each category and country?" we run 3 separate queries
#for each of those 3

#income
#market capitalization
#total assets

#min max and average of total income for each country
q = f"""
select ?name (avg(?v) as ?avg) (min(?v) as ?min) (max(?v) as ?max) where{{
?country <http://schema.org/name> ?name .
    {{
        select ?country ?v where{{
            ?company wdt:P31 wd:Q4830453 .
            ?company wdt:P3362 ?v .
            ?company wdt:P17 ?country .
        }}
    }}

}}
group by ?name
order by asc (?name)
"""
print("Income")
run_query(q)

Income
[('name', 'Argentina'), ('avg', '2180424362.333333333333333'), ('min', '-8920000'), ('max', '6549300000')]
[('name', 'Australia'), ('avg', '15996000000'), ('min', '15996000000'), ('max', '15996000000')]
[('name', 'Austria'), ('avg', '63277500'), ('min', '-457000000'), ('max', '424910000')]
[('name', 'Austria-Hungary'), ('avg', '-1637000'), ('min', '-1637000'), ('max', '-1637000')]
[('name', 'Austrian Empire'), ('avg', '-1637000'), ('min', '-1637000'), ('max', '-1637000')]
[('name', 'Belarus'), ('avg', '165383250'), ('min', '761000'), ('max', '486577000')]
[('name', 'Belgium'), ('avg', '4020000000'), ('min', '4020000000'), ('max', '4020000000')]
[('name', 'Bulgaria'), ('avg', '9808000'), ('min', '9808000'), ('max', '9808000')]
[('name', 'Canada'), ('avg', '1426000000'), ('min', '-149000000'), ('max', '2498000000')]
[('name', 'Cayman Islands'), ('avg', '34880992500'), ('min', '49985000'), ('max', '69712000000')]
[('name', 'Crimean Peninsula'), ('avg', '-488610000'), ('min', '-4886

49

In [51]:
#min max and average of total assets for each country
q = f"""
select ?name (avg(?v) as ?avg) (min(?v) as ?min) (max(?v) as ?max) where{{
?country <http://schema.org/name> ?name .
    {{
        select ?country ?v where{{
            ?company wdt:P31 wd:Q4830453 .
            ?company wdt:P2403 ?v .
            ?company wdt:P17 ?country .
        }}
    }}

}}
group by ?name
order by asc (?name)
"""
print("Total Assets")
run_query(q)

Total Assets
[('name', 'Algeria'), ('avg', '230000000000'), ('min', '230000000000'), ('max', '230000000000')]
[('name', 'Argentina'), ('avg', '436341125'), ('min', '436341125'), ('max', '436341125')]
[('name', 'Australia'), ('avg', '56002699549'), ('min', '12399098'), ('max', '111993000000')]
[('name', 'Austria'), ('avg', '10123721200'), ('min', '1490000000'), ('max', '32112000000')]
[('name', 'Austria-Hungary'), ('avg', '196766000'), ('min', '196766000'), ('max', '196766000')]
[('name', 'Austrian Empire'), ('avg', '196766000'), ('min', '196766000'), ('max', '196766000')]
[('name', 'Bahrain'), ('avg', '8467304000'), ('min', '8467304000'), ('max', '8467304000')]
[('name', 'Belarus'), ('avg', '1911946000'), ('min', '1911946000'), ('max', '1911946000')]
[('name', 'Belgium'), ('avg', '275200000000'), ('min', '275200000000'), ('max', '275200000000')]
[('name', 'Brazil'), ('avg', '68810000000'), ('min', '68810000000'), ('max', '68810000000')]
[('name', 'British Virgin Islands'), ('avg', '123

58

In [52]:
#min max and average of market capital for each country
q = f"""
select ?name (avg(?v) as ?avg) (min(?v) as ?min) (max(?v) as ?max) where{{
?country <http://schema.org/name> ?name .
    {{
        select ?country ?v where{{
            ?company wdt:P31 wd:Q4830453 .
            ?company wdt:P2226 ?v .
            ?company wdt:P17 ?country .
        }}
    }}

}}
group by ?name
order by asc (?name)
"""
print("Market capital")
run_query(q)

Market capital
[('name', 'Argentina'), ('avg', '602938304'), ('min', '602938304'), ('max', '602938304')]
[('name', 'Austria'), ('avg', '7230000000'), ('min', '3500000000'), ('max', '10960000000')]
[('name', 'Canada'), ('avg', '17810177452.666666666666667'), ('min', '648532358'), ('max', '45900000000')]
[('name', 'Czech Republic'), ('avg', '138800000000'), ('min', '7100000000'), ('max', '286200000000')]
[('name', 'Denmark'), ('avg', '142452000000'), ('min', '25256000000'), ('max', '289600000000')]
[('name', 'Finland'), ('avg', '220628500000'), ('min', '26257000000'), ('max', '415000000000')]
[('name', 'France'), ('avg', '30570379928.142857142857143'), ('min', '157659497'), ('max', '139000000000')]
[('name', 'Germany'), ('avg', '40829082512.5'), ('min', '111760000'), ('max', '139800000000')]
[('name', 'Greece'), ('avg', '3896025.7'), ('min', '3896025.7'), ('max', '3896025.7')]
[('name', 'India'), ('avg', '107711666666.666666666666667'), ('min', '4635000000'), ('max', '300500000000')]
[('

26

In [54]:
#we need to use several subqueries if we want to do everything in sparql

#get for each country: business and how many businesses it owns in other countries
q1 = f"""
select ?country ?company (count(*) as ?owned) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?country .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?country != ?anotherCountry) .
}}
group by ?country ?company
"""
#having q1, select for each returned country the greates amount of businesses owned in other countries across all companies
q2 = f"""
select ?country (max(?owned) as ?max) where{{

    {{
        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company



    }}
}}
group by ?country
"""
#this is a repeat of q1, with different names for the variables
q3 = f"""
select ?c ?ct (count(*) as ?tot) where{{
?c wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?ct .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?ct != ?anotherCountry) .
}}
group by ?c ?ct
"""

#having for each country the greatest number of businesses from another country that a certain local business owns select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



#and having the list of each business and the number of non local businesses it owns, along with its original country select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



#we simply compare country and count in the 2 sets, when they match we return the corresponding business
q4 = f"""
select ?countryName ?companyName ?max where{{
?c <http://schema.org/name> ?companyName .
?country <http://schema.org/name> ?countryName .
    {{
        select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



    }}
    
    {{
        select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



    }}
filter(?tot = ?max && ?ct = ?country) .
}}
"""
run_query(q4)

[('countryName', 'Sweden'), ('companyName', 'Electrolux'), ('max', '5')]
[('countryName', 'German Reich'), ('companyName', 'AG für Verkehrswesen'), ('max', '25')]
[('countryName', 'Australia'), ('companyName', 'BHP Group'), ('max', '1')]
[('countryName', 'Switzerland'), ('companyName', 'Richemont'), ('max', '4')]
[('countryName', 'Spain'), ('companyName', 'Telefónica'), ('max', '5')]
[('countryName', 'United States of America'), ('companyName', 'The Walt Disney Company'), ('max', '6')]
[('countryName', "People's Republic of China"), ('companyName', 'Alibaba Group'), ('max', '3')]
[('countryName', 'Russia'), ('companyName', 'Sberbank of Russia'), ('max', '2')]
[('countryName', 'Netherlands'), ('companyName', 'Heineken'), ('max', '10')]
[('countryName', 'Canada'), ('companyName', 'Corel'), ('max', '3')]
[('countryName', 'Ireland'), ('companyName', 'Accenture'), ('max', '1')]
[('countryName', 'Algeria'), ('companyName', 'Cevital'), ('max', '2')]
[('countryName', 'British Virgin Islands'),

90

In [62]:
#in the last query there are countries that looks like aren't existing anymore, let's look if also the companies have dissolved
#this was another property found earlier
dissolved = 'wdt:P576'
#let's check what values it has
q = f"""
select distinct ?state where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P576 ?state .
}}
limit 100
"""
run_query(q)

[('state', '2003-01-01T00:00:00Z')]
[('state', '1995-01-01T00:00:00Z')]
[('state', '1954-01-01T00:00:00Z')]
[('state', '1923-01-01T00:00:00Z')]
[('state', '2008-01-01T00:00:00Z')]
[('state', '2005-10-01T00:00:00Z')]
[('state', '2009-01-01T00:00:00Z')]
[('state', '1998-01-01T00:00:00Z')]
[('state', '2013-08-16T00:00:00Z')]
[('state', '2014-01-01T00:00:00Z')]
[('state', '1996-01-01T00:00:00Z')]
[('state', '1906-01-01T00:00:00Z')]
[('state', '1992-01-01T00:00:00Z')]
[('state', '1925-01-01T00:00:00Z')]
[('state', '1973-01-01T00:00:00Z')]
[('state', '2012-11-01T00:00:00Z')]
[('state', '2020-11-01T00:00:00Z')]
[('state', '1972-01-27T00:00:00Z')]
[('state', '1941-01-01T00:00:00Z')]
[('state', '1878-01-01T00:00:00Z')]
[('state', '1999-01-01T00:00:00Z')]
[('state', '2012-01-01T00:00:00Z')]
[('state', '1983-01-01T00:00:00Z')]
[('state', '2011-01-01T00:00:00Z')]
[('state', '2013-01-01T00:00:00Z')]
[('state', '1990-01-01T00:00:00Z')]
[('state', '2017-01-01T00:00:00Z')]
[('state', '2016-01-01T00:00

100

In [56]:
#dissolved indicates actually a date, probably the date in which a company was dissolved
#let's check if Microsoft was dissolved
q = f"""
select ?name where{{
wd:Q2283 <http://schema.org/name> ?name
filter not exists{{
        wd:Q2283 wdt:P576 ?date
    }}
}}
"""
run_query(q)

[('name', 'Microsoft')]


1

In [58]:
#let's try and see if blockbuster is present and was dissolved
q = f"""
select ?name ?dissolution where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?name ;
    wdt:P576 ?dissolution
filter regex(?name, 'blockbuster', 'i')
}}
"""
run_query(q)

[('name', 'Blockbuster LLC'), ('dissolution', '2014-01-12T00:00:00Z')]


1

In [59]:
#let's see the 10 companies that dissolved most recently
q = f"""
select ?name ?dissolution where{{
?company wdt:P31 wd:Q4830453 ;
    <http://schema.org/name> ?name ;
    wdt:P576 ?dissolution
}}
order by desc(?dissolution)
limit 10
"""
run_query(q)

[('name', 'Interactive Data Visualization Inc.'), ('dissolution', '2021-07-14T00:00:00Z')]
[('name', 'DB Dienstleistungen'), ('dissolution', 'nodeID://b383265')]
[('name', 'Saluka Investment'), ('dissolution', 'nodeID://b363939')]
[('name', 'Cespo Holdings'), ('dissolution', 'nodeID://b312546')]
[('name', "Dal Co'"), ('dissolution', 'nodeID://b308573')]
[('name', 'Nimura and Sato'), ('dissolution', 'nodeID://b273764')]
[('name', 'Goji Electronics'), ('dissolution', 'nodeID://b273278')]
[('name', 'Domus'), ('dissolution', 'nodeID://b244602')]
[('name', 'Bayer CropScience'), ('dissolution', '2022-01-01T00:00:00Z')]
[('name', 'Liberbank'), ('dissolution', '2021-07-30T00:00:00Z')]


10

In [60]:
#what are the top 10 countries that have seen the most companies dissolve?
q = f"""
select ?countryName (count(?dissolution) as ?howMany) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P576 ?dissolution ;
    wdt:P17 ?country .
?country <http://schema.org/name> ?countryName
}}
group by ?countryName
order by desc(?howMany)
limit 10
"""
run_query(q)

[('countryName', 'United States of America'), ('howMany', '1485')]
[('countryName', 'Germany'), ('howMany', '1235')]
[('countryName', 'Italy'), ('howMany', '820')]
[('countryName', 'United Kingdom'), ('howMany', '784')]
[('countryName', 'France'), ('howMany', '325')]
[('countryName', 'Norway'), ('howMany', '263')]
[('countryName', 'Czech Republic'), ('howMany', '230')]
[('countryName', 'Belgium'), ('howMany', '182')]
[('countryName', 'Sweden'), ('howMany', '159')]
[('countryName', 'Japan'), ('howMany', '148')]


10

In [61]:
#and finally, the top 10 countries for number of companies still active
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P17 ?country .
filter not exists {{
        ?company wdt:P576 ?dissolution
    }}
?country <http://schema.org/name> ?countryName
}}
group by ?countryName
order by desc(?howMany)
limit 10
"""
run_query(q)

[('countryName', 'United States of America'), ('howMany', '20819')]
[('countryName', 'Czech Republic'), ('howMany', '20579')]
[('countryName', 'Germany'), ('howMany', '10317')]
[('countryName', 'Japan'), ('howMany', '7254')]
[('countryName', 'United Kingdom'), ('howMany', '7023')]
[('countryName', 'Sweden'), ('howMany', '4115')]
[('countryName', 'Slovakia'), ('howMany', '3996')]
[('countryName', 'France'), ('howMany', '3445')]
[('countryName', 'Brazil'), ('howMany', '3334')]
[('countryName', 'Norway'), ('howMany', '3108')]


10

In [64]:
#get for each country: business and how many businesses it owns in other countries
#REPEAT WITHOUT DISSOLVED COMPANIES
q1 = f"""
select ?country ?company (count(*) as ?owned) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?country .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?country != ?anotherCountry) .
}}
group by ?country ?company
"""
#having q1, select for each returned country the greates amount of businesses owned in other countries across all companies
q2 = f"""
select ?country (max(?owned) as ?max) where{{

    {{
        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company



    }}
}}
group by ?country
"""
#this is a repeat of q1, with different names for the variables
q3 = f"""
select ?c ?ct (count(*) as ?tot) where{{
?c wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?ct .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?ct != ?anotherCountry) .
}}
group by ?c ?ct
"""

#having for each country the greatest number of businesses from another country that a certain local business owns select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



#and having the list of each business and the number of non local businesses it owns, along with its original country select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



#we simply compare country and count in the 2 sets, when they match we return the corresponding business
q4 = f"""
select ?countryName ?companyName ?max where{{
?c <http://schema.org/name> ?companyName .
?country <http://schema.org/name> ?countryName .
    {{
        select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



    }}
    
    {{
        select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



    }}
filter(?tot = ?max && ?ct = ?country) .
filter not exists {{

        ?c wdt:P576 ?dissolution .
    }}
}}
"""
run_query(q4)

[('countryName', 'Union of South Africa'), ('companyName', 'Union Corporation'), ('max', '1')]
[('countryName', 'France'), ('companyName', 'LVMH'), ('max', '10')]
[('countryName', 'United Kingdom'), ('companyName', 'Tesco'), ('max', '8')]
[('countryName', "People's Republic of China"), ('companyName', 'Alibaba Group'), ('max', '3')]
[('countryName', 'Brazil'), ('companyName', 'Anheuser-Busch InBev'), ('max', '9')]
[('countryName', 'Russia'), ('companyName', 'Sberbank of Russia'), ('max', '2')]
[('countryName', 'Russia'), ('companyName', 'Basic Element'), ('max', '2')]
[('countryName', 'Canada'), ('companyName', 'Corel'), ('max', '3')]
[('countryName', 'Japan'), ('companyName', 'Sony'), ('max', '10')]
[('countryName', 'United Kingdom of Great Britain and Ireland'), ('companyName', 'Brazilian Warrant Agency and Finance Co'), ('max', '1')]
[('countryName', 'Germany'), ('companyName', 'Aachen Münchener'), ('max', '7')]
[('countryName', 'Estonia'), ('companyName', 'Luminor Bank'), ('max', '

85

In [65]:
#get for each country: business and how many businesses it owns in other countries
#REPEAT WITHOUT DISSOLVED COMPANIES
#LIMITED TO CANADA, UK, US
q1 = f"""
select ?country ?company (count(*) as ?owned) where{{
?company wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?country .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?country != ?anotherCountry) .
}}
group by ?country ?company
"""
#having q1, select for each returned country the greates amount of businesses owned in other countries across all companies
q2 = f"""
select ?country (max(?owned) as ?max) where{{

    {{
        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company



    }}
}}
group by ?country
"""
#this is a repeat of q1, with different names for the variables
q3 = f"""
select ?c ?ct (count(*) as ?tot) where{{
?c wdt:P31 wd:Q4830453 ;
    wdt:P1830 ?anotherCompany ;
    wdt:P17 ?ct .
?anotherCompany wdt:P31 wd:Q4830453 ;
    wdt:P17 ?anotherCountry .
    
filter(?ct != ?anotherCountry) .
}}
group by ?c ?ct
"""

#having for each country the greatest number of businesses from another country that a certain local business owns select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



#and having the list of each business and the number of non local businesses it owns, along with its original country select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



#we simply compare country and count in the 2 sets, when they match we return the corresponding business
q4 = f"""
select ?countryName ?companyName ?max where{{
?c <http://schema.org/name> ?companyName .
?country <http://schema.org/name> ?countryName .
filter(?country in (wd:Q16, wd:Q145, wd:Q30))
    {{
        select ?c ?ct (count(*) as ?tot) where{{

?c wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?ct .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?ct != ?anotherCountry) .

}}

group by ?c ?ct



    }}
    
    {{
        select ?country (max(?owned) as ?max) where{{



    {{

        select ?country ?company (count(*) as ?owned) where{{

?company wdt:P31 wd:Q4830453 ;

    wdt:P1830 ?anotherCompany ;

    wdt:P17 ?country .

?anotherCompany wdt:P31 wd:Q4830453 ;

    wdt:P17 ?anotherCountry .

    

filter(?country != ?anotherCountry) .

}}

group by ?country ?company




    }}

}}

group by ?country



    }}
filter(?tot = ?max && ?ct = ?country) .
filter not exists {{

        ?c wdt:P576 ?dissolution .
    }}
}}
"""
run_query(q4)

[('countryName', 'United States of America'), ('companyName', 'The Walt Disney Company'), ('max', '6')]
[('countryName', 'Canada'), ('companyName', 'Corel'), ('max', '3')]
[('countryName', 'United States of America'), ('companyName', 'Ford Motor Company'), ('max', '6')]
[('countryName', 'United Kingdom'), ('companyName', 'Tesco'), ('max', '8')]


4